In [1]:
import pandas as pd

df = pd.DataFrame({
    "acctNo": ["123", "123", "123", "456", "789", "789"],
    "story": ["connected_persons", "connected_persons", "board_resolutions", 
              "connected_persons", "connected_persons", "board_resolutions"],
    "message": ["message A", "message B", "message C", "message A", "message B", "message C"],
    "risk": ["low risk", "low risk", "high risk", "low risk", "low risk", "high risk"],
})
df_cifs = pd.DataFrame({
    "cif": ["abc", "def", "ghi", "jkl"],
    "acctNo": ["123", "456", "789", "1011"]
})
df_unused = pd.DataFrame({
    "acctNo": ["123", "456", "789", "1011"],
    "unused": ["unused 1", "unused 2", "unused 3", "unused4"], 
})
# TODO: 2 cases, 1 is no text chunk extracted, 2 is no restrict text, can only tell from unused, both should be high risk

In [2]:
df_messages = df[["acctNo", "story", "message"]]
df_messages["risk"] = "warnings"
df_messages = df_messages.pivot_table(
    index = ["acctNo"], columns = ["story", "risk"], 
    values = ["message"], aggfunc = lambda x: "\n\n".join(x)
    )["message"]
df_messages

story,board_resolutions,connected_persons
risk,warnings,warnings
acctNo,,
123,message C,message A\n\nmessage B
456,NaN,message A
789,message C,message B


In [3]:
df_counts = pd.crosstab(index = [df.acctNo], columns = [df.story, df.risk], dropna = False)
df_counts

story  board_resolutions          connected_persons         
risk           high risk low risk         high risk low risk
acctNo                                                      
123                    1        0                 0        2
456                    0        0                 0        1
789                    1        0                 0        1

In [4]:
out = df_counts.merge(df_messages, on = "acctNo", how = "left")
out

story  board_resolutions          connected_persons           \
risk           high risk low risk         high risk low risk   
acctNo                                                         
123                    1        0                 0        2   
456                    0        0                 0        1   
789                    1        0                 0        1   

story  board_resolutions       connected_persons  
risk            warnings                warnings  
acctNo                                            
123            message C  message A\n\nmessage B  
456                  NaN               message A  
789            message C               message B

In [5]:
out = df_cifs.merge(out[out.columns.sort_values()], on = "acctNo", how = "left").merge(df_unused, on = "acctNo", how = "left")

/home/addison/anaconda3/envs/top2vec/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  validate=validate,


In [6]:
na_mapping = {}
for col in out.columns:
    if isinstance(col, tuple):
        if "high risk" in col:
            na_mapping[col] = 1
        if "low risk" in col:
            na_mapping[col] = 0
        if "warnings" in col:
            na_mapping[col] = "Message D" 

In [7]:
out = out.fillna(value = na_mapping)

In [8]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [9]:
wb = Workbook()
ws = wb.active

for idx, row in enumerate(dataframe_to_rows(out, index = False)):
    if idx == 0:
        headers = [e if isinstance(e, tuple) else (None, e) for e in row]
        headers = list(zip(*headers))
        for header in headers:
            ws.append(header)
        continue    
    ws.append(row)


for cell in ws["A"] + ws["B"] + ws[1] + ws[2]:
    cell.style = "Pandas"

ws.merge_cells('C1:E1')  
ws.merge_cells('F1:H1') 

In [10]:
wb.save("test.xlsx")

In [27]:
from inspect import getmembers, isfunction

class Category:
    category_one = "category_one"
    category_two = "category_two"

    def values():
        values = []
        for (name, value) in getmembers(Category):
            if not name.startswith("_"):
                if not isfunction(value): 
                    values.append(value)
        return values


In [28]:
print(Category.values())

['category_one', 'category_two']
